In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from joblib import load

In [2]:
typhi_grna = "./gRNA_sequences/typhi_grna.csv"
typhi_targets = "./gRNA_sequences/typhi_targets.csv"
typhi_records = pd.read_csv(typhi_grna)
typhi_records_targets = pd.read_csv(typhi_targets)
typhi_records['gRNA'] = typhi_records['gRNA'].str.lower()
typhi_records_targets['Sequence'] = typhi_records_targets['Sequence'].str.lower()
typhi_records.gRNA[0],typhi_records.Type[0],typhi_records_targets.Sequence[0],typhi_records_targets.Target[0]


('aaagataccagagcccgaat',
 'sty1607',
 'tttaaaagcatttgcggcagagaaaggacttgacggcgctgaaaaagatgcatttcttaagagtgcgtttgaacacttacatgcactaagtaaggctggtgaaccacttagtcttgaaacccttgttaatgccatttggccccaggctcccgaggagttatctggtaaacttgctgctgaagagttggagttatcggatggcttcgttcctgatggtcgagtcattcgggctctggtatcttttaaaggcaagtctaaatattgggaacttaagtttgaccgggaagggaaaactgaaggctatattgattatgatccggaaacaaatatt',
 'sty1607')

In [3]:
dict_base = {'a':'t','c':'g','t':'a','g':'c'}
dict_base['a']
def getComplement(target):
    comp = []
    for i in range(len(target)):
        comp.append(dict_base[target[-i-1]])
    return "".join(np.array(comp).ravel())
getComplement(typhi_records_targets.Sequence[0])

'aatatttgtttccggatcataatcaatatagccttcagttttcccttcccggtcaaacttaagttcccaatatttagacttgcctttaaaagataccagagcccgaatgactcgaccatcaggaacgaagccatccgataactccaactcttcagcagcaagtttaccagataactcctcgggagcctggggccaaatggcattaacaagggtttcaagactaagtggttcaccagccttacttagtgcatgtaagtgttcaaacgcactcttaagaaatgcatctttttcagcgccgtcaagtcctttctctgccgcaaatgcttttaaa'

In [4]:
#Find best alignment for gRNA against target 
def align(target, grna):
    max = 0
    comptarget = getComplement(target)
    comp = False
    #For every starting position of target
    for k in range(len(target)):
      count = 0
      countcomp = 0
      #For every nucleotide of the gRNA sequence 
      for l in range(min(len(target[k:]),len(grna))):
        if target[k+l] == grna[l]:
          count += 1
        if comptarget[k+l] == grna[l]:
          countcomp += 1
      if count >= max:
        max = count
        position = k
      if countcomp >= max:
        max = countcomp
        position = k
        comp = True
    if position < 20 or position + 40 > len(target):
      return None,0

    if comp:
      return comptarget[position-20:position+40],max

    return target[position-20:position+40],max

In [5]:
for k in range(53):
    selected = False
    for i in range(len(typhi_records_targets)):
        target = typhi_records_targets.Sequence[i]
        if typhi_records.gRNA[k] in target or typhi_records.gRNA[k] in getComplement(target):
            selected = True
    print(selected)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
targets = []
hamming = []
for k in range(len(typhi_records)):
    target,max = align(typhi_records_targets[typhi_records_targets['Target']==typhi_records.Type[k]]['Sequence'].tolist()[0],typhi_records.gRNA[k])
    
    targets.append(target)
    hamming.append(20-max)

In [7]:
np.array(hamming)

array([ 0,  1,  0,  0, 20,  0, 20,  0, 20,  0,  0,  0,  0,  0, 20, 20,  0,
        0, 20, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 20,  0])

In [8]:
#Import features from Random Forest Model
clf = load('Models/grnamodel_ds.joblib')
output_ds = pd.read_pickle("./Outputs/df_feat_ds_doubletree.pkl") 
output_ds = output_ds[output_ds.importance_values>0.01]
output_ds

,features,importance_values
690,Motif: ta,0.079278
625,gc_target,0.040530
258,"targetposition41, g",0.024618
647,Motif: c,0.022447
685,Motif: gta,0.019166
634,Motif: acc,0.018614
627,Motif: aa,0.018292
668,Motif: g,0.023565
653,Motif: cc,0.016772
629,Motif: aac,0.015493


In [9]:
def one_hot_encode(seq):
    mapping = dict(zip("acgt", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]
def one_hot_encode_duplets(seq):
    mapping = dict(zip(('aa', 'ac', 'ag', 'at', 'ca', 'cc', 'cg', 'ct', 'ga', 'gc', 'gg', 'gt', 'ta', 'tc', 'tg', 'tt'), range(16)))
    seq2 = []
    for i in range(len(seq)-1):
      seq2.append(mapping[seq[i:i+2]])
    return np.eye(16)[seq2]
def calcGC(seq):
  count = 0
  for l in range(len(seq)):
    if seq[l] == 'g' or seq[l] == 'c':
      count += 1
  return count/len(seq)
def countMotifs(seq, motifs):
  counts = []
  for motif in motifs:
    count = 0
    for i in range(len(seq) - len(motif)):
      if seq[i:i+len(motif)] == motif:
        count += 1
    counts.append(count/(len(seq) - len(motif)))
  return counts
bases = ['a','c','g','t']
motifs_to_look_for = []
for i in range(len(bases)):
  motifs_to_look_for.append(bases[i])
  for j in range(len(bases)):
    motifs_to_look_for.append(bases[i]+bases[j])
    for k in range(len(bases)):
      motifs_to_look_for.append(bases[i]+bases[j]+bases[k])
      # for l in range(len(bases)):
      #   motifs_to_look_for.append(bases[i]+bases[j]+bases[k]+bases[l])
motifs_to_look_for.append('aaaa')
motifs_to_look_for.append('cccc')
motifs_to_look_for.append('gggg')
motifs_to_look_for.append('tttt')

In [10]:
X_ds = []
true_typhi_records = []
true_typhi_type = []
true_typhi_hamming = []
true_typhi_target = []
k = 20
for i in range(len(typhi_records)):
  ## 1 is not length 20 so ignore
  if len(typhi_records.gRNA[i])!=20:
    continue
  # if no suitable target, ignore
  if targets[i] == None:
    continue
  else: 
    true_typhi_records.append(typhi_records.gRNA[i])
    true_typhi_type.append(typhi_records.Type[i])
    true_typhi_hamming.append(hamming[i])
    true_typhi_target.append(targets[i])
  t = np.concatenate((one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(targets[i][:k]), one_hot_encode(targets[i][k+20:]), one_hot_encode_duplets(typhi_records.gRNA[i])
                           , calcGC(typhi_records.gRNA[i]), calcGC(targets[i]), countMotifs(typhi_records.gRNA[i], motifs_to_look_for)), axis = None)
  X_ds.append(t)
X_reduced_ds = np.array(X_ds)[:,output_ds.index]

In [11]:
y_pred = clf.predict(X_reduced_ds)
y_pred

array([1.25777088, 1.04771158, 1.93127595, 1.26744404, 1.51545617,
       1.40852941, 1.44864287, 1.43174234, 1.50398035, 0.99776969,
       1.30873587, 1.63493786, 1.66490719, 1.43545085, 1.02340051,
       2.11489438, 1.47893429, 1.44867257, 0.98929268, 0.98300813,
       1.758941  , 1.42790552, 1.56214253, 1.15805115, 1.30340737,
       1.23113298, 1.28376037, 1.25149406, 1.8540474 , 1.18317106,
       1.12621474, 1.77862381, 1.8305046 , 1.13218802, 1.87426277,
       1.5744292 , 1.59396368, 1.39298906, 1.50768478, 1.61011738,
       0.63271167, 0.77055827, 1.01754702, 0.87805564, 1.73705249,
       1.13463955, 1.32109434])

In [12]:
for i in range(len(y_pred)):
    if true_typhi_hamming[i]!=0:
        y_pred[i] = 0

In [15]:
df = pd.DataFrame({'gRNA':true_typhi_records, 'my version of target':true_typhi_target, 'type':true_typhi_type, 'hamming distance':true_typhi_hamming,'predicted_ds_k':y_pred})
df = df.sort_values('predicted_ds_k', ascending = False)
df = df.reset_index(drop = True)
df.to_csv('./Outputs/typhi_kvalues_prediction.csv')
df

,gRNA,my version of target,type,hamming distance,predicted_ds_k
0,ccgcgcttgagaataggtga,gttgaaaatgttgagatttcccgcgcttgagaataggtgaagtagc...,sty2879,0,2.114894
1,gacttgcctttaaaagatac,acttaagttcccaatatttagacttgcctttaaaagataccagagc...,sty1607,0,1.931276
2,attaatgccacgctcatacg,taaacttcacacctgatttaattaatgccacgctcatacgggccgc...,sty0307,0,1.874263
3,tatgcaggcattggttatga,gtgtgaatgacatactgccccggcactggtcggctcaatcagatcg...,sty0307,0,1.854047
4,aaggccgtcttctcccgata,tttttattacccagttctgtctgaaatttagtcggatcacgccaga...,sty0307,0,1.830505
5,aaaatacctgtgcggcccgt,atatgactaccactcccaccattatatccatcaattttccagaaga...,sty0307,0,1.778624
6,gtagcttactacgccgattg,taggtgaagtagctctttagatcacgggtgatacgactacggctcg...,sty2879,0,1.758941
7,ttacccagttctgtctgaaa,agaagaccacaccttttttattacccagttctgtctgaaatttagt...,sty0307,0,1.737052
8,agtgtaatctggcagcttta,ggcaaaaaaatcactttttaagtgtaatctggcagctttaagatgc...,sty2879,0,1.664907
9,agtcaacgagccgtagtcgt,cacacacttagcttgagatcatgatgtttgttcaatcggcgtagta...,sty2879,0,1.634938
